In [37]:
from collections import defaultdict
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.stats import mannwhitneyu,kruskal,ttest_ind,f_oneway
import plotly.io as pio
pio.kaleido.scope.default_format = "pdf"
import plotly.express as px
from matplotlib.colors import ListedColormap
import matplotlib.colors as mc
import matplotlib.pyplot as plt
import scikit_posthocs as sp
import itertools
import seaborn as sns
from itertools import product

In [49]:
A_tad = pd.read_csv('./results/A_2_TADs.bed',sep='\t',usecols=[0,1,2])
B_tad = pd.read_csv('./results/S_2_TADs.bed',sep='\t',usecols=[0,1,2])
borders_type = pd.read_csv('../data/Agglo_borders_type.bed',sep='\t',header=None)

In [39]:
def which_types(tad_region,up_dict,down_dict):
    #print(tad_region)
    keys = up_dict.keys()
    for key_pairs in list(product(keys,repeat=2)):
        if (np.isin(tad_region['start'],up_dict[key_pairs[0]])) & (np.isin(tad_region['end'],down_dict[key_pairs[1]])):
            return key_pairs

In [40]:
vwhich_types = np.vectorize(which_types)

In [41]:
def extract_tad_type(borders_type,tad,agglo_type,chrom):
    borders_chr = borders_type.loc[borders_type.iloc[:,0]==chrom,:]
    tad_chr = tad.loc[tad['chrom']==chrom,:]
    upper_dict = defaultdict(list)
    for index, row in borders_chr.iterrows():
        upper_dict[int(row[3])].append(int(row[2]))
    down_dict = defaultdict(list)
    for index, row in borders_chr.iterrows():
        down_dict[int(row[3])].append(int(row[1]))
    tad_type = list(tad_chr.apply(lambda x: which_types(x.loc[['start','end']],upper_dict,down_dict),axis=1,result_type='reduce'))
    tad_chr_type = tad_chr
    tad_chr_type['type'] = list(map(lambda t: agglo_type+''.join(map(str,t)),tad_type))
    return tad_chr_type

In [52]:
result_list = []
for chrom in A_tad['chrom'].unique():
    result_list.append(extract_tad_type(borders_type,A_tad,'A',chrom))
A_type = pd.concat(result_list)

/home/data4/jh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [54]:
A_type.to_csv('./data/A2_type.bed',sep='\t',quoting=3,header=True,index=False)

In [35]:
result_list = []
for chrom in A_tad['chrom'].unique():
    result_list.append(extract_tad_type(borders_type,A_tad,'A',chrom))
A_type = pd.concat(result_list)

/home/data4/jh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [36]:
A_type.to_csv('./data/A_type.bed',sep='\t',quoting=3,header=True,index=False)

In [48]:
borders_type

,chr1,4440000,4480000,1
0,chr1,4800000,4840000,0
1,chr1,6200000,6240000,1
2,chr1,9520000,9560000,1
3,chr1,10160000,10200000,1
4,chr1,11600000,11640000,1
...,...,...,...,...
2131,chrX,164480000,164520000,1
2132,chrX,166160000,166200000,1
2133,chrX,167280000,167320000,1
2134,chrX,168640000,168680000,1
